<a href="https://colab.research.google.com/github/BAEK-SEUNGYUN/modu_deep/blob/main/modu_wine_check_stop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 학습의 자동 중단

학습이 진행될수록 학습셋의 정확도는 올라가지만 과적합 때문에 테스트셋의 실험 결과는 점점 나빠진다.

케라스에는 이렇게 학습이 진행되어도 테스트셋 오차가 줄지 않으면 학습을 멈추게 하는 함수가 있는데 -> **EarlyStopping()** 함수이다.

```
from keras.callbacks import EarlyStopping
```

EarlyStopping() 함수에 모니터할 값과 테스트 오차가 좋아지지 않아도 몇 번까지 기다릴지 정한다.

이를 early_stopping_callback에 저장

```
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)
```

앞서 정한 그대로 에포크 획수와 배치 크기 등을 설정하고 early_stopping_callback 값을 불러온다.

```
model.fit(X,Y, validation_split=0.33, epochs = 2000, batch_size=500, callbacks=[early_stopping_calback
```

모두 코드에 적용해보자


In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

import pandas as pd
import numpy as np
import tensorflow as tf

In [4]:
# seed 값 설정
np.random.seed(3)
tf.random.set_seed(3)

In [5]:
df_pre = pd.read_csv('/content/drive/MyDrive/dataset/wine.csv', header=None)
df = df_pre.sample(frac=0.15)

In [6]:
dataset = df.values
dataset

array([[ 5.9  ,  0.18 ,  0.28 , ..., 10.65 ,  7.   ,  0.   ],
       [10.2  ,  0.67 ,  0.39 , ..., 10.   ,  5.   ,  1.   ],
       [ 8.4  ,  0.715,  0.2  , ...,  9.4  ,  5.   ,  1.   ],
       ...,
       [ 5.8  ,  0.2  ,  0.34 , ..., 11.7  ,  5.   ,  0.   ],
       [ 5.8  ,  0.33 ,  0.2  , ...,  8.9  ,  5.   ,  0.   ],
       [ 6.2  ,  0.47 ,  0.21 , ...,  9.2  ,  5.   ,  0.   ]])

In [7]:
X = dataset[:, 0:12]
Y = dataset[:, 12]

In [14]:
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [15]:
model.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])

In [16]:
# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

In [17]:
# 모델 실행
model.fit(X, Y, validation_split=0.2, epochs = 2000,
          batch_size=500, callbacks=[early_stopping_callback])

Epoch 1/2000
2/2 [==============================] - 1s 330ms/step - loss: 9.3318 - accuracy: 0.2749 - val_loss: 8.8269 - val_accuracy: 0.2256
Epoch 2/2000
2/2 [==============================] - 0s 24ms/step - loss: 8.0149 - accuracy: 0.2776 - val_loss: 7.3845 - val_accuracy: 0.2256
Epoch 3/2000
2/2 [==============================] - 0s 24ms/step - loss: 6.7022 - accuracy: 0.2696 - val_loss: 6.0765 - val_accuracy: 0.2256
Epoch 4/2000
2/2 [==============================] - 0s 26ms/step - loss: 5.4046 - accuracy: 0.2776 - val_loss: 5.3788 - val_accuracy: 0.2256
Epoch 5/2000
2/2 [==============================] - 0s 31ms/step - loss: 4.7959 - accuracy: 0.2709 - val_loss: 4.8094 - val_accuracy: 0.2256
Epoch 6/2000
2/2 [==============================] - 0s 24ms/step - loss: 4.2231 - accuracy: 0.2782 - val_loss: 4.0741 - val_accuracy: 0.2256
Epoch 7/2000
2/2 [==============================] - 0s 26ms/step - loss: 3.5569 - accuracy: 0.2729 - val_loss: 3.2250 - val_accuracy: 0.2256
Epoch 8/2000

epochs를 2000으로 설정했지만 627번만에 끝나버렸다.

In [12]:
#결과 출력
print("\n Accuracy : %.4f" % (model.evaluate(X,Y)[1]))

31/31 [==============================] - 0s 1ms/step - loss: 0.0432 - accuracy: 0.9867

 Accuracy : 0.9867


모델 업데이트 함수와 학습 자동중단 함수를 동시에 사용해보자

In [25]:
from keras.callbacks import ModelCheckpoint

In [20]:
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [21]:
model.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])

In [22]:
# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
  os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:.4f}.hdf5"

In [26]:
# 모델 업데이트 및 저장
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss',
                               verbose = 1, save_best_only=True)

In [27]:
# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

In [28]:
model.fit(X, Y, validation_split=0.2, epochs = 3500,
          batch_size=500, callbacks=[early_stopping_callback])

Epoch 1/3500
2/2 [==============================] - 1s 343ms/step - loss: 2.1001 - accuracy: 0.7251 - val_loss: 1.2069 - val_accuracy: 0.7744
Epoch 2/3500
2/2 [==============================] - 0s 23ms/step - loss: 1.5713 - accuracy: 0.7224 - val_loss: 0.8604 - val_accuracy: 0.7744
Epoch 3/3500
2/2 [==============================] - 0s 22ms/step - loss: 1.0754 - accuracy: 0.7304 - val_loss: 0.5777 - val_accuracy: 0.7744
Epoch 4/3500
2/2 [==============================] - 0s 25ms/step - loss: 0.7344 - accuracy: 0.7207 - val_loss: 0.7209 - val_accuracy: 0.6256
Epoch 5/3500
2/2 [==============================] - 0s 24ms/step - loss: 0.7390 - accuracy: 0.5978 - val_loss: 0.8500 - val_accuracy: 0.4615
Epoch 6/3500
2/2 [==============================] - 0s 26ms/step - loss: 0.7672 - accuracy: 0.5388 - val_loss: 0.5361 - val_accuracy: 0.7333
Epoch 7/3500
2/2 [==============================] - 0s 26ms/step - loss: 0.5686 - accuracy: 0.7112 - val_loss: 0.4439 - val_accuracy: 0.7692
Epoch 8/3500

전체 샘플의 15%만 사용으로 780개의 샘플이 입력

epochs가 3500번 이었으나 1197번까지만 돌았습니다.